In [1]:
# import sqlite3

# # Connect to the database
# conn = sqlite3.connect('data.db')
# cursor = conn.cursor()

# # Add the new column BLM to the podcast_episodes table
# cursor.execute('ALTER TABLE podcast_episodes ADD COLUMN BLM INTEGER DEFAULT 0')

# # Commit the changes and close the connection
# conn.commit()
# cursor.close()
# conn.close()

In [1]:
import sqlite3
from tabulate import tabulate

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the SELECT query to fetch the first 5 rows from the podcast_episodes table
cursor.execute('SELECT * FROM podcast_episodes LIMIT 3')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection
cursor.close()
conn.close()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import sqlite3
import numpy as np
from gensim import corpora, models

In [ ]:
import sqlite3
import numpy as np
from gensim import corpora, models

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Fetch the tokens from the database
cursor.execute("SELECT rowid, tokens FROM podcast_episodes WHERE tokens IS NOT NULL")
rows = cursor.fetchall()

# Prepare the corpus from tokens
texts = []
row_ids = []

for row in rows:
    row_id, token_str = row
    # Convert the stored token strings back to lists (assuming tokens were stored as space-separated text)
    tokens = token_str.split()
    texts.append(tokens)
    row_ids.append(row_id)

# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the LDA model
num_topics = 100
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Get the dominant topic for each episode
dominant_topics = []
for bow in corpus:
    topic_dist = lda_model.get_document_topics(bow)
    if topic_dist:
        dominant_topic = max(topic_dist, key=lambda x: x[1])[0]  # Topic with highest weight
        dominant_topics.append(dominant_topic)
    else:
        dominant_topics.append(None)

# Ensure the 'topic' column exists in the table
cursor.execute("PRAGMA table_info(podcast_episodes);")
columns = [info[1] for info in cursor.fetchall()]
if 'topic' not in columns:
    print("Adding 'topic' column to the table.")
    cursor.execute("ALTER TABLE podcast_episodes ADD COLUMN topic INTEGER")

# Update the database with the dominant topic
for row_id, topic in zip(row_ids, dominant_topics):
    if topic is not None:
        cursor.execute("UPDATE podcast_episodes SET topic = ? WHERE rowid = ?", (topic, row_id))

# Commit and close
conn.commit()
conn.close()

print(f"Updated {len(dominant_topics)} episodes with their dominant topics.")